# DS Primer - Week 2 - Data Management

## Socio-Political Analysis on Human Development Index

### 1. Importing and Loading File - Country Events Dataset

In order to get the various armed conflict events happenning in a country, we need to find a source where we can find the events in countries in addition to the particular year on which an event occurs.
The Uppasala Conflict Data Program dashboard would be a great source for this data - https://ucdp.uu.se/

We have already consolidated and downloaded a dataset from this datasource and you can find it in `Datasets/Dataset_Country_Events.csv` path

In [137]:
import pandas as pd
import numpy as np

country_events = pd.read_csv('Datasets/Dataset_Country_Events.csv')

In [144]:
country_events.head()

,year,active_year,type_of_violence,conflict_new_id,conflict_name,dyad_new_id,dyad_name,side_a_new_id,gwnoa,side_a,...,date_prec,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,low,high
0,2010,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2010-09-25,2010-09-25,2,0,0,0,2,2,2
1,2011,1,3,715,Government of Yemen (North Yemen) - Civilians,1182,Government of Yemen (North Yemen) - Civilians,123,678.0,Government of Yemen (North Yemen),...,1,2011-02-19,2011-02-19,0,0,0,0,0,0,2
2,2011,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2011-04-16,2011-04-16,0,0,0,0,0,0,1
3,2012,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2012-06-04,2012-06-04,4,1,0,0,5,5,5
4,2012,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2012-12-09,2012-12-09,0,4,1,0,5,5,5


##### Data Granularity 

Here, it seems that each row represent a particular event that occured in a country

### 2. Data Preparation

In [139]:
country_events = country_events.drop(['id'], axis=1)
country_events = country_events[country_events.year.isin(['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015'])]

In [142]:
# iterrows() function is used for iterating through the rows in pandas
# strip() function is used to trim the string
# we are trimming the extra spaces in country column
for index, row in country_events.iterrows():
    row['country'] = (row['country']).strip()
    row['year'] = (row['country']).strip()      

#### 2.1 Data Normalization

We are planning to integrate the socio-politcal and the human development index data. For that you need to normalize the data so that you can merge the data across multiple data sources

In [143]:
# While looking at the country column it seems the names are more elaborate and would be difficult to join the data based on countries with other datasets
# So changing the name of the country names to a standard format
country_events.loc[country_events['country'] == 'Yemen (North Yemen)', 'country'] = 'Yemen'
country_events.loc[country_events['country'] == 'DR Congo (Zaire)', 'country'] = 'DR Congo'
country_events.loc[country_events['country'] == 'Madagascar (Malagasy)', 'country'] = 'Madagascar'
country_events.loc[country_events['country'] == 'Zimbabwe (Rhodesia)', 'country'] = 'Zimbabwe'
country_events.loc[country_events['country'] == 'Serbia (Yugoslavia)', 'country'] = 'Serbia'
country_events.loc[country_events['country'] == 'Cambodia (Kampuchea)', 'country'] = 'Cambodia'
country_events.loc[country_events['country'] == 'Russia (Soviet Union)', 'country'] = 'Russia'
country_events.loc[country_events['country'] == 'Myanmar (Burma)', 'country'] = 'Myanmar'

In [79]:
# For a common and uniform format let's convert the datatype of the column year from integer to a string
country_events['year'] = country_events['year'].apply(str)
country_events['country'] = country_events['country'].apply(str)

# Exporting the dataframe into a csv
country_events.to_csv('news_article_conflicts.csv')

#### 2.2 Data Aggregation

The data granularity of the dataset is every individual event that had happened in a country. 
But for a master dataset we might need aggregated count for a particular country

In [145]:
conflict_aggregate = country_events.groupby(['country','year'], as_index=False).agg({'best': 'sum', 'deaths_civilians': 'sum'})
conflict_aggregate

,country,year,best,deaths_civilians
0,Afghanistan,2005,1711,200
1,Afghanistan,2006,4982,477
2,Afghanistan,2007,7020,669
3,Afghanistan,2008,5660,763
4,Afghanistan,2009,6499,1099
...,...,...,...,...
507,Zimbabwe,2007,0,0
508,Zimbabwe,2008,253,253
509,Zimbabwe,2009,0,0
510,Zimbabwe,2011,1,1


In [146]:
# Renaming the columns
conflict_aggregate=conflict_aggregate.rename(columns = {'best':'total_deaths'})
conflict_aggregate=conflict_aggregate.rename(columns = {'deaths_civilians':'civilian_deaths'})

In [147]:
conflict_aggregate

,country,year,total_deaths,civilian_deaths
0,Afghanistan,2005,1711,200
1,Afghanistan,2006,4982,477
2,Afghanistan,2007,7020,669
3,Afghanistan,2008,5660,763
4,Afghanistan,2009,6499,1099
...,...,...,...,...
507,Zimbabwe,2007,0,0
508,Zimbabwe,2008,253,253
509,Zimbabwe,2009,0,0
510,Zimbabwe,2011,1,1


In [155]:
# Saving the aggregated file as csv
conflict_aggregate.to_csv('conflict_aggregate.csv')

### 2. Importing and Loading File - Human Development Index Data

The data source for this particular file can be found here -http://hdr.undp.org/en/content/human-development-index-hdi
The explanation of HDI will also be available here

In [156]:
#Forming the HDI data
import pandas as pd
hdi_data_payload = pd.read_csv('Datasets/Dataset_HDI_Data.csv',encoding = "ISO-8859-1")
hdi_data = hdi_data_payload[['Country','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]

In [157]:
hdi_data.head()

,Country,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afghanistan,0.405,0.415,0.433,0.434,0.448,0.454,0.463,0.470,0.476,0.479,0.479
1,Albania,0.696,0.703,0.713,0.721,0.725,0.738,0.752,0.759,0.761,0.762,0.764
2,Algeria,0.686,0.690,0.697,0.705,0.714,0.724,0.732,0.737,0.741,0.743,0.745
3,Andorra,NaN,NaN,NaN,NaN,NaN,0.819,0.819,0.843,0.850,0.857,0.858
4,Angola,0.439,0.454,0.468,0.480,0.488,0.495,0.508,0.523,0.527,0.531,0.533


#### 2.1 Data Manipulation 
One of the important steps and needed steps in the industry

In [158]:
# here we will invert the axes and manipulate data such that country and year become a row value
hdi_data = hdi_data.melt(id_vars=['Country'])
hdi_data

,Country,variable,value
0,Afghanistan,2005,0.405
1,Albania,2005,0.696
2,Algeria,2005,0.686
3,Andorra,2005,NaN
4,Angola,2005,0.439
...,...,...,...
2063,Venezuela (Bolivarian Republic of),2015,0.767
2064,Viet Nam,2015,0.683
2065,Yemen,2015,0.482
2066,Zambia,2015,0.579


In [159]:
# Renaming and Cleaning the dataset
hdi_data=hdi_data.rename(columns = {'Country':'country'})
hdi_data=hdi_data.rename(columns = {'variable':'year'})

for index, row in hdi_data.iterrows():
    row['country'] = (row['country']).strip()
    row['year'] = (row['year']).strip()

#### 2. Data Normalization

In [160]:
hdi_data.loc[hdi_data['country'] == 'Bolivia (Plurinational State of)', 'country'] = 'Bolivia'
hdi_data.loc[hdi_data['country'] == 'Bosnia and Herzegovina', 'country'] = 'Bosnia-Herzegovina'
hdi_data.loc[hdi_data['country'] == 'Congo (Democratic Republic of the)', 'country'] = 'DR Congo'
hdi_data.loc[hdi_data['country'] == 'Côte d\'Ivoire', 'country'] = 'Ivory Coast'
hdi_data.loc[hdi_data['country'] == 'Iran (Islamic Republic of)', 'country'] = 'Iran'
hdi_data.loc[hdi_data['country'] == 'Lao People\'s Democratic Republic', 'country'] = 'Laos'
hdi_data.loc[hdi_data['country'] == 'Moldova (Republic of)', 'country'] = 'Moldova'
hdi_data.loc[hdi_data['country'] == 'Venezuela (Bolivarian Republic of)', 'country'] = 'Venezuela'
hdi_data.loc[hdi_data['country'] == 'Tanzania (United Republic of)', 'country'] = 'Tanzania'
hdi_data.loc[hdi_data['country'] == 'The former Yugoslav Republic of Macedonia', 'country'] = 'Macedonia'
hdi_data.loc[hdi_data['country'] == 'Viet Nam', 'country'] = 'Vietnam'

In [161]:
hdi_data.to_csv('hdi_master.csv')

#### 3. Data Merging

In [162]:
hdi_data

,country,year,value
0,Afghanistan,2005,0.405
1,Albania,2005,0.696
2,Algeria,2005,0.686
3,Andorra,2005,NaN
4,Angola,2005,0.439
...,...,...,...
2063,Venezuela,2015,0.767
2064,Vietnam,2015,0.683
2065,Yemen,2015,0.482
2066,Zambia,2015,0.579


In [164]:
print(type(conflict_aggregate.country[0]))
print(type(conflict_aggregate.year[0]))
print(type(hdi_data.country[0]))
print(type(hdi_data.year[0]))

<class 'str'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>


In [165]:
conflict_aggregate['year'] = conflict_aggregate.year.astype(str)

In [168]:
a = 2

In [170]:
str(a)

'2'

In [ ]:
country_events['year'] = country_events.year.apply(str)

In [166]:
# We now try to merge the country events data with the HDI data
master_data = pd.merge(conflict_aggregate, hdi_data,how="left", on=['country', 'year'])
master_data.to_csv('conflict_hdi.csv')

In [167]:
master_data

,country,year,total_deaths,civilian_deaths,value
0,Afghanistan,2005,1711,200,0.405
1,Afghanistan,2006,4982,477,0.415
2,Afghanistan,2007,7020,669,0.433
3,Afghanistan,2008,5660,763,0.434
4,Afghanistan,2009,6499,1099,0.448
...,...,...,...,...,...
507,Zimbabwe,2007,0,0,0.421
508,Zimbabwe,2008,253,253,0.419
509,Zimbabwe,2009,0,0,0.436
510,Zimbabwe,2011,1,1,0.464


## Political Data about Countries

In [121]:
#Political Data for EDA
political_data = pd.read_csv('Datasets/Dataset_Democracy_Data.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,7,2150,3307,3323,3324,3361,3394,3444,3446,3448,3449,3451,3452,3453,3455,3456,3457,3459,3460,3461,3463,3464,3465,3467,3468,3469,3471,3472,3474,3475) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [122]:
political_data

,country_name,country_id,country_text_id,year,historical_date,codingstart,gapstart,gapend,codingend,COWcode,...,e_pemaliex,e_pefeliex,e_peheight,e_miurbpop,e_miurbani,e_Vanhanen_nonag_ipo,e_Vanhanen_urban_ipo,e_miinteco,e_miinterc,e_Civil_War
0,Afghanistan,36,AFG,1900,1900-12-31,1900-01-01,NaN,NaN,2016-12-31,700.0,...,NaN,NaN,NaN,170238.406250,0.032186,NaN,NaN,0.0,0.0,NaN
1,Afghanistan,36,AFG,1901,1901-12-31,1900-01-01,NaN,NaN,2016-12-31,700.0,...,NaN,NaN,NaN,173962.371875,0.032618,NaN,NaN,0.0,0.0,NaN
2,Afghanistan,36,AFG,1902,1902-12-31,1900-01-01,NaN,NaN,2016-12-31,700.0,...,NaN,NaN,NaN,177686.337500,0.033043,NaN,NaN,0.0,0.0,NaN
3,Afghanistan,36,AFG,1903,1903-12-31,1900-01-01,NaN,NaN,2016-12-31,700.0,...,NaN,NaN,NaN,181410.303125,0.033462,NaN,NaN,0.0,0.0,NaN
4,Afghanistan,36,AFG,1904,1904-12-31,1900-01-01,NaN,NaN,2016-12-31,700.0,...,NaN,NaN,NaN,185134.268750,0.033873,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17599,Zanzibar,236,ZZB,2012,2012-12-31,1900-01-01,NaN,NaN,2016-12-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17600,Zanzibar,236,ZZB,2013,2013-12-31,1900-01-01,NaN,NaN,2016-12-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17601,Zanzibar,236,ZZB,2014,2014-12-31,1900-01-01,NaN,NaN,2016-12-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17602,Zanzibar,236,ZZB,2015,2015-12-31,1900-01-01,NaN,NaN,2016-12-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
political_data = political_data[political_data.year.isin(['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015'])]

In [124]:
# This particular step involves a lot of domain analysis. We will survey the columns in the data dictionary and then selecting the respective columns=
politics_data =political_data[['country_name','year','v2x_polyarchy','v2x_liberal','v2x_partipdem','v2x_egaldem','v2x_frassoc_thick','v2x_freexp','v2xme_altinf','v2x_suffr','v2xel_frefair','v2x_elecoff',
                               'v2xcl_rol','v2x_jucon','v2x_cspart','v2xeg_eqprotec','v2xeg_eqdr','v2xcs_ccsi','v2x_gender','v2x_gencl','v2x_elecreg','v2xel_elecpres','v2x_corr','v2x_civlib','v2elrstrct','v2elcomvot','v2elrsthos',
                                'v2elmulpar_ord','v2psparban_ord','v2psbars_ord','v2ex_elechos','v2clacfree_ord','v2clrelig_ord','v2cltort_ord','v2clkill_ord','v2cltrnslw_ord','v2clfmove_ord','v2cldiscw_ord','v2clstown_ord','v2xcl_acjst','v2clacjust_ord','v2svinlaut_ord',
                                'v2csreprss_ord','v2csrlgrep_ord','v2mecenefm_ord','v2mecenefi_ord','v2mebias_ord','v2mecorrpt_ord','v2pepwrses_ord','v2pepwrgen_ord','v2peedueq_ord','v2pehealth_ord','v2peprisch','v2pesecsch','e_v2x_regime',
                                'e_boix_regime','e_sovereignty','e_fh_cl','e_fh_pair','e_SF1','e_SF3','e_Vanhanen_literate_ipo','e_regionpol','e_regiongeo','e_migdppc','e_migdpgro','e_miinflat','e_reserves_billions','e_peinfmor','e_pelifeex']]

In [125]:
politics_data=politics_data.rename(columns = {'country_name':'country'})

In [126]:
politics_data.loc[politics_data['country'] == 'Cape Verde', 'country'] = 'Cabo Verde'
politics_data.loc[politics_data['country'] == 'Democratic Republic of Congo', 'country'] = 'DR Congo'
politics_data.loc[politics_data['country'] == 'Republic of the Congo', 'country'] = 'Congo'
politics_data.loc[politics_data['country'] == 'SÃ£o TomÃ© och PrÃ­ncipe', 'country'] = 'Sao Tome and Principe'

In [127]:
politics_data['country'] = politics_data['country'].apply(str)
politics_data['year'] = politics_data['year'].apply(str)

#### Data Merging - Political and HDI data

In [132]:
poltics_master = pd.merge(politics_data, hdi_data, on=['country', 'year'])

#### Data Cleaning

In [130]:
polity_eda = poltics_master.dropna(thresh=0.8*len(poltics_master), axis=1)

In [133]:
print(polity_eda.isnull().sum())

country                0
year                   0
v2x_polyarchy          1
v2x_liberal            0
v2x_partipdem          1
v2x_egaldem            1
v2x_frassoc_thick      0
v2x_freexp             0
v2xme_altinf           0
v2x_suffr              0
v2xel_frefair          0
v2x_elecoff            0
v2xcl_rol              0
v2x_jucon              0
v2x_cspart             0
v2xeg_eqprotec         0
v2xeg_eqdr             0
v2xcs_ccsi             0
v2x_gender            25
v2x_gencl              0
v2x_elecreg            0
v2xel_elecpres         0
v2x_corr               0
v2x_civlib            11
v2elrstrct             3
v2elcomvot             1
v2elrsthos             2
v2psparban_ord         0
v2psbars_ord           0
v2ex_elechos           0
v2clacfree_ord         0
v2clrelig_ord          0
v2cltort_ord           0
v2clkill_ord           0
v2cltrnslw_ord         0
v2clfmove_ord          0
v2cldiscw_ord          0
v2clstown_ord          0
v2xcl_acjst            0
v2clacjust_ord         0


In [134]:
# Imputing mean values for Numerical Variables
polity_eda['v2x_polyarchy'].fillna((polity_eda['v2x_polyarchy'].mean()), inplace=True)
polity_eda['v2x_partipdem'].fillna((polity_eda['v2x_partipdem'].mean()), inplace=True)
polity_eda['v2x_egaldem'].fillna((polity_eda['v2x_egaldem'].mean()), inplace=True)
polity_eda['v2x_gender'].fillna((polity_eda['v2x_gender'].mean()), inplace=True)
polity_eda['v2x_civlib'].fillna((polity_eda['v2x_civlib'].mean()), inplace=True)
polity_eda['value'].fillna((polity_eda['value'].mean()), inplace=True)

In [135]:
# Imputing mode values for Categorical Variables
polity_eda['v2elrstrct'].fillna((polity_eda['v2elrstrct'].mode()[0]), inplace=True)
polity_eda['v2elcomvot'].fillna((polity_eda['v2elcomvot'].mode()[0]), inplace=True)
polity_eda['v2elrsthos'].fillna((polity_eda['v2elrsthos'].mode()[0]), inplace=True)
polity_eda['v2csreprss_ord'].fillna((polity_eda['v2csreprss_ord'].mode()[0]), inplace=True)
polity_eda['v2csrlgrep_ord'].fillna((polity_eda['v2csrlgrep_ord'].mode()[0]), inplace=True)
polity_eda['v2mecenefm_ord'].fillna((polity_eda['v2mecenefm_ord'].mode()[0]), inplace=True)
polity_eda['v2mecenefi_ord'].fillna((polity_eda['v2mecenefi_ord'].mode()[0]), inplace=True)
polity_eda['v2mebias_ord'].fillna((polity_eda['v2mebias_ord'].mode()[0]), inplace=True)
polity_eda['v2mecorrpt_ord'].fillna((polity_eda['v2mecorrpt_ord'].mode()[0]), inplace=True)
polity_eda['e_fh_pair'].fillna((polity_eda['e_fh_pair'].mode()[0]), inplace=True)
polity_eda['e_regionpol'].fillna((polity_eda['e_regionpol'].mode()[0]), inplace=True)
polity_eda['e_regiongeo'].fillna((polity_eda['e_regiongeo'].mode()[0]), inplace=True)

In [136]:
# polity_eda.to_csv('polity_data.csv')
polity_eda.to_csv('polity_master.csv')